In [ ]:
import os
import random
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import (
    BertTokenizerFast,
    BertForQuestionAnswering,
    AdamW,
    get_linear_schedule_with_warmup,
    DataCollatorWithPadding
)
import time
from tqdm import tqdm
import re
import string

In [ ]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [ ]:
# device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
import json
# Load training data
with open('Spoken-SQuAD/spoken_train-v1.1.json', 'r') as f:
    training_data = json.load(f)
# Load validation data
with open('Spoken-SQuAD/spoken_test-v1.1.json', 'r') as f:
    validation_data = json.load(f)


In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
def prepare_examples_with_stride(data, tokenizer, max_length=512, stride=128):
    examples = []
    for article in data['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answers = qa.get('answers', [])
                
                # Skip examples without answers
                if not answers:
                    continue

                answer = answers[0]
                answer_text = answer['text']
                start_char = answer['answer_start']
                end_char = start_char + len(answer_text)
            
                tokenized_inputs = tokenizer(
                    question,
                    context,
                    max_length=max_length,
                    truncation='only_second',
                    stride=stride,
                    return_overflowing_tokens=True,
                    return_offsets_mapping=True,
                    padding='max_length'
                )

                # Map tokens
                overflow_mapping = tokenized_inputs.pop('overflow_to_sample_mapping')
                offset_mapping = tokenized_inputs.pop('offset_mapping')

                for i in range(len(tokenized_inputs['input_ids'])):
                    input_ids = tokenized_inputs['input_ids'][i]
                    attention_mask = tokenized_inputs['attention_mask'][i]
                    token_type_ids = tokenized_inputs['token_type_ids'][i]
                    sequence_ids = tokenized_inputs.sequence_ids(i)

                    context_start = sequence_ids.index(1)
                    context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

                    offsets = offset_mapping[i]
                    sample_index = overflow_mapping[i]

                    
                    if not (offsets[context_start][0] <= start_char and offsets[context_end][1] >= end_char):
                       
                        start_positions = 0
                        end_positions = 0
                    else:
                        # Finding the start and end token indices
                        token_start_index = context_start
                        token_end_index = context_end

                        while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                            token_start_index += 1
                        start_positions = token_start_index - 1

                        while token_end_index >= 0 and offsets[token_end_index][1] >= end_char:
                            token_end_index -= 1
                        end_positions = token_end_index + 1

                    # example
                    example = {
                        'input_ids': input_ids,
                        'attention_mask': attention_mask,
                        'token_type_ids': token_type_ids,
                        'start_positions': start_positions,
                        'end_positions': end_positions,
                        'question': question,
                        'context': context,
                        'answers': [answer_text]
                    }
                    examples.append(example)
    return examples

# Preprocess the data
training_examples = prepare_examples_with_stride(training_data, tokenizer)
validation_examples = prepare_examples_with_stride(validation_data, tokenizer)

print(f"Total training examples: {len(training_examples)}")
print(f"Total validation examples: {len(validation_examples)}")


Total training examples: 37130
Total validation examples: 5376


In [7]:
class QADataset(Dataset):
    def __init__(self, examples):
        self.examples = examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return {
            'input_ids': torch.tensor(example['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(example['attention_mask'], dtype=torch.long),
            'token_type_ids': torch.tensor(example['token_type_ids'], dtype=torch.long),
            'start_positions': torch.tensor(example['start_positions'], dtype=torch.long),
            'end_positions': torch.tensor(example['end_positions'], dtype=torch.long)
        }


In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Creating datasets
train_dataset = QADataset(training_examples)
val_dataset = QADataset(validation_examples)

# Creating DataLoaders
batch_size = 8 

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator
)


In [ ]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
model.to(device)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)
epochs = 3 
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

In [ ]:
def train(model, train_dataloader, val_dataloader, optimizer, scheduler, epochs, device, output_dir):
    total_start_time = time.time()
    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(epochs):
        print(f"\n======== Epoch {epoch + 1}/{epochs} ========")
        print("Training...")

        epoch_start_time = time.time()
        total_loss = 0.0
        model.train()

        for step, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", leave=False)):
            optimizer.zero_grad()
            batch = {k: v.to(device) for k, v in batch.items()}

            with torch.cuda.amp.autocast():
                outputs = model(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    token_type_ids=batch['token_type_ids'],
                    start_positions=batch['start_positions'],
                    end_positions=batch['end_positions']
                )
                loss = outputs.loss

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            batch_loss = loss.item()
            total_loss += batch_loss

            # Print batch loss every 500 steps
            if (step + 1) % 500 == 0 or (step + 1) == len(train_dataloader):
                current_lr = scheduler.get_last_lr()[0]
                elapsed = time.time() - epoch_start_time
                print(f"  Step {step + 1}/{len(train_dataloader)} - "
                      f"Batch Loss: {batch_loss:.4f} - "
                      f"Avg Loss: {total_loss / (step + 1):.4f} - "
                      f"LR: {current_lr:.6f} - "
                      f"Elapsed: {elapsed:.2f}s")

        avg_train_loss = total_loss / len(train_dataloader)
        epoch_time = time.time() - epoch_start_time
        print(f"\nEpoch {epoch + 1} finished.")
        print(f"  Average Training Loss: {avg_train_loss:.4f}")
        print(f"  Epoch Training Time: {epoch_time:.2f}s")

        # Validation after each epoch
        print("\nRunning Validation...")
        val_start_time = time.time()
        avg_val_loss = evaluate(model, val_dataloader, device)
        val_time = time.time() - val_start_time
        print(f"  Validation Loss: {avg_val_loss:.4f}")
        print(f"  Validation Time: {val_time:.2f}s")

        # Computing evaluation metrics
        print("\nCalculating Metrics on Validation Set...")
        evaluate_metrics(model, validation_examples, device)

        # Saving model after each epoch
        epoch_output_dir = os.path.join(output_dir, f"epoch_{epoch + 1}")
        if not os.path.exists(epoch_output_dir):
            os.makedirs(epoch_output_dir)
        model.save_pretrained(epoch_output_dir)
        tokenizer.save_pretrained(epoch_output_dir)
        print(f"Model saved to {epoch_output_dir}")

    total_training_time = time.time() - total_start_time
    print(f"\nTraining complete! Total training time: {total_training_time:.2f}s")


In [ ]:
def evaluate(model, dataloader, device):
    model.eval()
    total_eval_loss = 0.0

    for step, batch in enumerate(tqdm(dataloader, desc="Evaluating", leave=False)):
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.no_grad():
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                token_type_ids=batch['token_type_ids'],
                start_positions=batch['start_positions'],
                end_positions=batch['end_positions']
            )
            loss = outputs.loss

        total_eval_loss += loss.item()

        if (step + 1) % 100 == 0 or (step + 1) == len(dataloader):
            print(f"  Evaluation Step {step + 1}/{len(dataloader)} - "
                  f"Loss: {loss.item():.4f}")

    avg_eval_loss = total_eval_loss / len(dataloader)
    return avg_eval_loss


In [14]:
def normalize_answer(s):
    """Lowercase, remove punctuation, articles, and extra whitespace."""
    def lower(text):
        return text.lower()
    def remove_punctuation(text):
        return ''.join(ch for ch in text if ch not in set(string.punctuation))
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    return white_space_fix(remove_articles(remove_punctuation(lower(s))))

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_tokens = normalize_answer(a_gold).split()
    pred_tokens = normalize_answer(a_pred).split()
    common = set(gold_tokens) & set(pred_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gold_tokens)
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1


In [ ]:
def evaluate_metrics(model, examples, device):
    model.eval()
    exact_scores = []
    f1_scores = []

    for idx, example in enumerate(tqdm(examples, desc='Calculating Metrics')):
        # Tokenize with offset mapping and overflow mapping
        tokenized_inputs = tokenizer(
            example['question'],
            example['context'],
            truncation='only_second',
            max_length=512,
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding='max_length',
            return_tensors='pt'
        )

        offset_mapping = tokenized_inputs.pop('offset_mapping')
        overflow_mapping = tokenized_inputs.pop('overflow_to_sample_mapping')

        inputs = {k: v.to(device) for k, v in tokenized_inputs.items()}

        with torch.no_grad():
           
            outputs = model(**inputs)

        # Postprocessing to get the best answer from multiple chunks
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        all_answers = []
        for i in range(start_logits.size(0)):
            start_logit = start_logits[i]
            end_logit = end_logits[i]
            input_ids = inputs['input_ids'][i]
            offsets = offset_mapping[i]

            start_indexes = torch.argsort(start_logit, descending=True).tolist()[:20]
            end_indexes = torch.argsort(end_logit, descending=True).tolist()[:20]

            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Ensure that the end index is after the start index and the answer is not too long
                    if start_index <= end_index and end_index - start_index + 1 <= 30:
                        # Calculate the score for the answer span
                        answer_score = start_logit[start_index] + end_logit[end_index]
                        # Skip answers that are not in the context (sequence_ids=1)
                        if inputs['token_type_ids'][i][start_index] != 1 or inputs['token_type_ids'][i][end_index] != 1:
                            continue
                        # Extracting answers using offsets
                        start_char = offsets[start_index][0].item()
                        end_char = offsets[end_index][1].item()
                        predicted_answer = example['context'][start_char:end_char]
                        all_answers.append((predicted_answer, answer_score))

        if all_answers:
            best_answer = max(all_answers, key=lambda x: x[1])[0]
        else:
            best_answer = ''

        # Ground truth answers
        ground_truth_answers = example['answers']

        
        exact = max(compute_exact(gt, best_answer) for gt in ground_truth_answers)
        f1 = max(compute_f1(gt, best_answer) for gt in ground_truth_answers)

        exact_scores.append(exact)
        f1_scores.append(f1)

        if (idx + 1) % 500 == 0 or (idx + 1) == len(examples):
            avg_exact = sum(exact_scores) / len(exact_scores)
            avg_f1 = sum(f1_scores) / len(f1_scores)
            print(f"  Processed {idx + 1}/{len(examples)} examples - "
                  f"EM: {avg_exact:.4f} - F1: {avg_f1:.4f}")

    avg_exact = sum(exact_scores) / len(exact_scores)
    avg_f1 = sum(f1_scores) / len(f1_scores)

    print(f"\nFinal Exact Match (EM): {avg_exact:.4f}")
    print(f"Final F1 Score: {avg_f1:.4f}")


In [ ]:
def predict_answers(model, questions, contexts, tokenizer, device):
    model.eval()
    answers = []

    for question, context in zip(questions, contexts):
        tokenized_inputs = tokenizer(
            question,
            context,
            truncation='only_second',
            max_length=512,
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding='max_length',
            return_tensors='pt'
        )
        offset_mapping = tokenized_inputs.pop('offset_mapping')
        overflow_mapping = tokenized_inputs.pop('overflow_to_sample_mapping')

      
        inputs = {k: v.to(device) for k, v in tokenized_inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        all_answers = []
        for i in range(start_logits.size(0)):
            start_logit = start_logits[i]
            end_logit = end_logits[i]
            input_ids = inputs['input_ids'][i]
            offsets = offset_mapping[i]
            token_type_ids = inputs['token_type_ids'][i]

            start_indexes = torch.argsort(start_logit, descending=True).tolist()[:20]
            end_indexes = torch.argsort(end_logit, descending=True).tolist()[:20]

            for start_index in start_indexes:
                for end_index in end_indexes:
                   
                    if start_index <= end_index and end_index - start_index + 1 <= 30:
                        
                        if token_type_ids[start_index] != 1 or token_type_ids[end_index] != 1:
                            continue
                        
                        answer_score = start_logit[start_index] + end_logit[end_index]
                        
                        start_char = offsets[start_index][0].item()
                        end_char = offsets[end_index][1].item()
                        predicted_answer = context[start_char:end_char]
                        all_answers.append((predicted_answer, answer_score))
                        
        if all_answers:
            best_answer = max(all_answers, key=lambda x: x[1])[0]
        else:
            best_answer = ''

        answers.append(best_answer)

    return answers

In [17]:
output_dir = './models/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [18]:
train(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    optimizer=optimizer,
    scheduler=scheduler,
    epochs=epochs,
    device=device,
    output_dir=output_dir
)


/local_scratch/slurm.995155/ipykernel_1527704/1203427822.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



======== Epoch 1/3 ========
Training...


Epoch 1:   0%|          | 0/4642 [00:00<?, ?it/s]/local_scratch/slurm.995155/ipykernel_1527704/1203427822.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1:  11%|█         | 503/4642 [00:24<03:05, 22.30it/s]

  Step 500/4642 - Batch Loss: 2.2808 - Avg Loss: 4.3370 - LR: 0.000011 - Elapsed: 24.06s


Epoch 1:  22%|██▏       | 1004/4642 [00:47<02:43, 22.30it/s]

...

  Processed 4500/5376 examples - EM: 0.5502 - F1: 0.6957


Calculating Metrics:  93%|█████████▎| 5010/5376 [01:48<00:07, 47.67it/s]

  Processed 5000/5376 examples - EM: 0.5526 - F1: 0.6977


Calculating Metrics: 100%|██████████| 5376/5376 [01:56<00:00, 46.32it/s]


  Processed 5376/5376 examples - EM: 0.5506 - F1: 0.6962

Final Exact Match (EM): 0.5506
Final F1 Score: 0.6962
Model saved to ./models/epoch_3

Training complete! Total training time: 1110.87s


In [ ]:
# Trial 
questions = ["What is the capital of France?"]
contexts = ["France, officially the French Republic, is a country whose territory consists of metropolitan France in Western Europe and several overseas regions and territories. The capital is Paris."]
predicted_answers = predict_answers(model, questions, contexts, tokenizer, device)

for question, answer in zip(questions, predicted_answers):
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    
    

Question: What is the capital of France?
Answer: Paris
